# Kalkulačka

Jako první si musíme promyslet celkové rozvržení naši kalkulačky. <br>
Tahle kalkulačka bude mít jedno textové pole, kde se bude zapisovat a po sléze vypisovat výsledek (využijeme `wx.TextCtrl`).
<br> Pod textem vytvoříme pole tlačítek 4x4, kam dáme 0-9, se základními operacemi. <br> Nakonec přidáme tlačítka pro " Clear " a výsledek " = ". <br>
![main](img/kal1.png)
Nyní k samotnému kódu.

In [1]:
import wx

class MyFrame(wx.Frame):
    def __init__(self, parent, title, size):
        super().__init__(parent, title =title, size = size)
        self.panel = MyPanel(self)
        self.SetSizeHints(350, 375)

class MyPanel(wx.Panel):
    def __init__(self, parent):
        super().__init__(parent)
        self.Gui()

    def Gui(self):
        mainsizer = wx.BoxSizer(wx.VERTICAL)

        self.textpanel = wx.TextCtrl(self, style=wx.TE_RIGHT)
        mainsizer.Add(self.textpanel, 0, wx.EXPAND|wx.ALL , 5)
        
        btnsizer = wx.GridSizer(4,4,10,10)
        buttons = ['7', '8', '9', '/','4', '5', '6', '*','1', '2', '3', '-', '.', '0', 'C', '+']
        for label in buttons:
            btnsizer.Add(wx.Button(self, label=label), 1, wx.EXPAND)
        mainsizer.Add(btnsizer, 2, wx.ALL | wx.EXPAND, 5)
        
        Sizer = wx.BoxSizer(wx.HORIZONTAL)

        self.equal = wx.Button(self, label='=')
        self.clearbtn = wx.Button(self, label='Clear') 
        Sizer.Add(self.clearbtn, 1, wx.ALL | wx.EXPAND, 5)
        Sizer.Add(self.equal, 1, wx.ALL | wx.EXPAND, 5)
        mainsizer.Add(Sizer, 1, wx.EXPAND)
        self.SetSizer(mainsizer)
        
        self.Bind(wx.EVT_BUTTON, self.onpress)

    def onpress(self, event):
        label = event.GetEventObject().GetLabel()
        clearerror = self.textpanel.GetValue()

        if clearerror == "Error":
            self.textpanel.SetValue("")

        if label == "Clear":
            self.textpanel.SetValue("")
            
        elif label == "=":
            try:
                rawresult = self.textpanel.GetValue()
                result = eval(rawresult)
                self.textpanel.SetValue(str(result))
            except:
                self.textpanel.SetValue("Error")
                
        elif label == "C":
            removelast = self.textpanel.GetValue()
            removed = removelast[:-1]
            self.textpanel.SetValue(removed)
            
        else:
            self.textpanel.AppendText(label)

class MyApp(wx.App):
    def OnInit(self):
        self.frame = MyFrame(parent=None, title="Kalkulačka", size = (350, 375))
        self.frame.Show()
        return True

if __name__ == "__main__": 
    app=MyApp()
    app.MainLoop()

### GUI

Pro GUI kalkulačky vytvoříme samostatnou funkci ve třídě `wx.Panel`. <br>

Vytvoříme si hlavní vertikální `BoxSizer`, do kterého vložíme všechny naše prvky. <br>
Stejně tak si vytvoříme náš textový panel `wx.TextCtrl` který uložíme do proměnné se `self`, protože ji budem používat v jiných funkcích. V parametrech `wx.TextCtrl` nastavíme styl, aby se text zobrazoval na pravé straně. <br>
Nakonec přidáme prvek do sizeru. S parametry proporce na `0`, aby neměnil svoji výšku. Nastavíme `flag` na `wx.EXPAND`. Nastavíme i ohraničení ze všech stran s `wx.ALL`, které napíšeme ke flagu s `|` (roura), posledním číslem je velikost odstazení v pixelech.

In [ ]:
        mainsizer = wx.BoxSizer(wx.VERTICAL)

        self.textpanel = wx.TextCtrl(self, style=wx.TE_RIGHT)
        mainsizer.Add(self.textpanel, 0, wx.EXPAND|wx.ALL , 5)

Nyní si vytvoříme seznam se všemi tlačítky. Vytvořili jsme si jich 16, abychom udělali pole s 4x4 tlačítky. <br>
Vytvoříme si náš `GridSizer` a poté pomocí funkce `for`vytvoříme všech 16 tlačítek, které přímo přidáme do sizeru. <br>
![GridSizer](img/kal2.png)
Poté náš nově vytvořený sizer dáme do hlavního sizeru. S Parametry podobnými jako předešle až na parametr proporce, kde nastavíme `2` (protože budeme chtít, aby sizer zabíral více místa než poslední sizer, který si teprve vytvoříme).

In [ ]:
        btnsizer = wx.GridSizer(4,4,10,10)
        buttons = ['7', '8', '9', '/','4', '5', '6', '*','1', '2', '3', '-', '.', '0', 'C', '+']
        for label in buttons:
            btnsizer.Add(wx.Button(self, label=label), 1, wx.EXPAND)
        mainsizer.Add(btnsizer, 2, wx.EXPAND|wx.ALL, 5)

Teďka přidáme další 2 zbývající tlačítka pro " = " a " Clear ". Přidáme je opět do sizeru. Proporci u obou nastavíme na `1`, aby při roztahování zabírali stejné místo. Sizer přidáme do hlavního sizeru s proporcí `1`. Zobrazíme hlavní sizer.

In [ ]:
        Sizer = wx.BoxSizer(wx.HORIZONTAL)

        self.equal = wx.Button(self, label='=')
        self.clearbtn = wx.Button(self, label='Clear') 
        Sizer.Add(self.clearbtn, 1, wx.ALL | wx.EXPAND, 5)
        Sizer.Add(self.equal, 1, wx.ALL | wx.EXPAND, 5)
        
        mainsizer.Add(Sizer, 1, wx.EXPAND)
        self.SetSizer(mainsizer)

Nakonec přidáme metodu pro bindování tlačítek. Více v části Logiky.

In [ ]:
        self.Bind(wx.EVT_BUTTON, self.onpress)

### Logika
V `GridSizeru` jsme si ulehčili práci s vytvářením tlačítek pomocí cyklu `for`, avšak nevýhodou je, že tlačítka nejsou uložené v proměnných. <br>
Proto celá logika programu bude, že při stisknutí tlačítka se zjistí jeho `label` a podle tlačítko zareaguje.

Logika tlačítek je taková, že si buď vytáhneme hodnotu `labelu` tlačítka a podle zmáčklého tlačítka se přidá hodnota nebo se spustí speciální event. <br>
Speciálně definujeme tlačítka "Clear", "=" a "C". <br>
Logika výpočtu samotného vyřešíme jednodušše pomocí metody `eval()`, která převádí (podobně jako `int()` nebo `str()`) datový typ stringu do "python výrazu" (expression). Např: `eval("2 * 3")` outputem bude `6`. Samotná metoda je velmi komplexní a nedoporučuje se používat v programech s přístupem do OS. Další využití najdete [zde](https://www.programiz.com/python-programming/methods/built-in/eval)<br>

Nyní teda k praxi. Label stisklého tlačítka vytáhneme pomocí `GetLabel()`, ale protože konkrétní tlačítka nemáme definované , použiujeme metodu `event.GetEventObject()`, jedná se o metodu [`wx.Eventu`](https://wxpython.org/Phoenix/docs/html/wx.Event.html), kdy nám tahle funkce zjistí tlačítko, které jsme stistkli.

In [ ]:
label = event.GetEventObject().GetLabel()

Samotná podmínka má 5 částí: <br>
1. zjištění zda text obsahuje "Error" a jeho následné smazání v panelu
2. funkce pro tlačítko " Clear ", které smaže text v panelu
3. samotná vyhodnocující funkce, kdy jsme využili `eval()` i pro kontrolu vstupu uživatele => Pokud bude v kalkulačce zadané neplatný či nesmyslný výpočet, tak funkce `eval()` vyhodí error. Využili jsme [Try a Except](https://www.w3schools.com/python/python_try_except.asp). Je důležité si ještě ohlídat, že `eval()` vrací hodnotu v `int`, ale v text panelu zadáváme `str`
4. funkce pro tlačítko smazání posledního charakteru.
5. do else jsme vložili funkci přidání text pro tlačítka 0-9, +, -, * a /.

In [ ]:
        clearerror = self.textpanel.GetValue()
        if clearerror == "Error":
            self.textpanel.SetValue("")

        if label == "Clear":
            self.textpanel.SetValue("")
            
        elif label == "=":
            try:
                rawresult = self.textpanel.GetValue()
                result = eval(rawresult)
                self.textpanel.SetValue(str(result))
            except:
                self.textpanel.SetValue("Error")
                
        elif label == "C":
            removelast = self.textpanel.GetValue()
            removed = removelast[:-1]
            self.textpanel.SetValue(removed)
            
        else:
            self.textpanel.AppendText(label)